In [1]:
# import os
# os.listdir('../input/')
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/IDS 576/Project/Final Project (1)')

In [3]:
!pwd

/content/drive/My Drive/IDS 576/Project/Final Project (1)


In [4]:
from python_utils import *
import time
import matplotlib.pyplot as plt
import cv2 as cv
from math import sqrt 
import pandas as pd
import numpy as np
from torchvision import transforms as tfs
import torch
from PIL import Image
from torch.autograd import Variable
from torchvision.utils import save_image
import pickle
import random
import argparse
import sys
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import ion, show
import warnings
warnings.filterwarnings('ignore')

In [ ]:
time_start = time.time()
time_start

In [ ]:
datas = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Project/fer2013/fer2013.csv")
Data_RNN = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Project/fer2013/fer2013.csv")
datas

In [ ]:
lab = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neural']

## For RNN ## 

lab_RNN = ['This person is feeling very angry', 'This person is feeling Disgust', 
           'This person is feeling scared', 'This person is feeling Happy',
           'This person is feeling sad', 'This person is extremely surprised',
           'This person has no emotions']

def set_value(row_number, assigned_value): 
    return assigned_value[row_number] 
  
# Create the dictionary 
event_dictionary = {0 : 'This person is feeling very angry', 1 : 'This person is feeling Disgust',
                   2 : 'This person is feeling scared', 3 : 'This person is feeling Happy',
                   4 : 'This person is feeling sad', 5 : 'This person is extremely surprised',
                   6 : 'This person has no emotions'} 
  

# Add a new column named 'Price' 
Data_RNN['lab_RNN'] = Data_RNN['emotion'].apply(set_value, args =(event_dictionary, ))

labels_num = datas.emotion.value_counts()
la = [0,1,2,3,4,5,6]
la_num = [labels_num[i] for i in range(len(labels_num))]
print(labels_num)
plt.bar(range(len(la_num)), la_num,color='rgbc',tick_label=lab) 
for a,b in zip(la,la_num):  
    plt.text(a, b+0.05, '%.0f' % b, ha='center', va= 'bottom',fontsize=10)  
plt.show() 

Data_RNN

In [ ]:
sets = datas.Usage.value_counts()
da = [sets[i] for i in range(len(sets))]
set_la = ['Training','PublicTest','PrivateTest']
print(sets)
plt.axes(aspect=1)
plt.title('Size of Training,PublicTest,PrivateTest sets in the image dataset')
plt.pie(x = da,labels = set_la,autopct='%3.1f %%', shadow=True)
plt.show()

In [ ]:
print('Picture Length:',len(datas.pixels[1].split()))

Picture Length: 2304


In [ ]:
time_1 = time.time()
print('Time to read：',round((time_1 - time_start),2),'s')

In [ ]:
train_set = datas[(datas.Usage == 'Training')] 
val_set = datas[(datas.Usage == 'PublicTest')]
test_set = datas[(datas.Usage == 'PrivateTest')] 
X_train = np.array(list(map(str.split, train_set.pixels)), np.float32)
X_val = np.array(list(map(str.split, val_set.pixels)), np.float32) 
X_test = np.array(list(map(str.split, test_set.pixels)), np.float32) 
X_train = X_train.reshape(X_train.shape[0], 48, 48) 
X_val = X_val.reshape(X_val.shape[0],48,48) 
X_test = X_test.reshape(X_test.shape[0],48, 48) 

2.Tag Data Processing

In [ ]:
y_train = list(train_set.emotion) 
y_val = list(val_set.emotion)
y_test = list(test_set.emotion )

3.Checking some samples

In [ ]:
fig = plt.figure(figsize = (10,8))
for i in range(len(X_train[:35])):
    if(y_train[i] == 0 ):
        str_la = 'Angry'
        img = Image.fromarray(np.uint8(X_train[i]))
    elif(y_train[i] == 1):
        str_la = 'Disgust'
        img = Image.fromarray(np.uint8(X_train[i]))
    elif(y_train[i] == 2):
        str_la = 'Fear'
        img = Image.fromarray(np.uint8(X_train[i]))
    elif(y_train[i] == 3):
        str_la = 'Happy'
        img = Image.fromarray(np.uint8(X_train[i]))
    elif(y_train[i] == 4):
        str_la = 'Sad'
        img = Image.fromarray(np.uint8(X_train[i]))
    elif(y_train[i] == 5):
        str_la = 'Surprise'
        img = Image.fromarray(np.uint8(X_train[i]))
    elif(y_train[i] == 6):
        str_la = 'Neural'
        img = Image.fromarray(np.uint8(X_train[i]))
    y = fig.add_subplot(5,7,i+1)
    y.imshow(img,cmap='gray')
    plt.title(str_la)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

4.Data augmentation

In [ ]:
train_preprocess = tfs.Compose([
    tfs.ToPILImage(),
    tfs.RandomCrop(44),
    tfs.RandomHorizontalFlip(),
    tfs.ToTensor(),
])



val_preprocess = tfs.Compose([
    tfs.ToPILImage(),
    tfs.TenCrop(44),
    tfs.Lambda(lambda crops: torch.stack([tfs.ToTensor()(crop) for crop in crops])),
])

In [ ]:
import csv
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader

batch_size= 128
class DataLr:

    def __init__(self, data):

        data_train = data[data.Usage == 'Training']
        data_validation = data[data.Usage == 'PrivateTest']
        data_Test = data[data.Usage == 'PublicTest']

        self.Y_train = data_train['emotion']
        X_train = data_train['pixels']

        self.Y_test = data_Test['emotion']
        self.X_test = data_Test['pixels']
        X_train = X_train.str.split(" ", n=2304, expand=True)
        self.X_train = X_train.astype(np.int)

        self.train_samples = X_train.shape[0]


class DataSetFactory:

    def __init__(self):
        images = []
        emotions = []
        private_images = []
        private_emotions = []
        public_images = []
        public_emotions = []

        with open("/content/drive/My Drive/Colab Notebooks/Project/fer2013/fer2013.csv", 'r') as csvin:
            data = csv.reader(csvin)
            next(data)
            for row in data:
                face = [int(pixel) for pixel in row[1].split()]
                face = np.asarray(face).reshape(48, 48)
                face = face.astype('uint8')

                if row[-1] == 'Training':
                    emotions.append(int(row[0]))
                    images.append(Image.fromarray(face))
                elif row[-1] == "PrivateTest":
                    private_emotions.append(int(row[0]))
                    private_images.append(Image.fromarray(face))
                elif row[-1] == "PublicTest":
                    public_emotions.append(int(row[0]))
                    public_images.append(Image.fromarray(face))

        print('training size %d : private val size %d : public val size %d' % (
            len(images), len(private_images), len(public_images)))
        train_transform = transforms.Compose([transforms.ToTensor()])
        
        val_transform = transforms.Compose([transforms.ToTensor()])
        
        test_transform = transforms.Compose([transforms.ToTensor()])
        
        self.training = DataSet(transform=train_transform, images=images, emotions=emotions)
        self.private = DataSet(transform=val_transform, images=private_images, emotions=private_emotions)
        self.public = DataSet(transform=test_transform, images=public_images, emotions=public_emotions)

class DataSet(torch.utils.data.Dataset):

    def __init__(self, transform=None, images=None, emotions=None):
        self.transform = transform
        self.images = images
        self.emotions = emotions

    def __getitem__(self, index):
        image = self.images[index]
        emotion = self.emotions[index]
        if self.transform is not None:
            image = self.transform(image)
        return image, emotion

    def __len__(self):
        return len(self.images)


# DataLoaders provide various ways to get batches of examples.

def loaders(Data, batch_size):
    Data = DataSetFactory()
    train_loader = torch.utils.data.DataLoader(Data.training, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(Data.private, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(Data.public, batch_size=batch_size, shuffle=True)
    return train_loader, val_loader, test_loader


Data = DataSetFactory()

train_loader, val_loader, test_loader = loaders(Data, batch_size)

training size 28709 : private val size 3589 : public val size 3589
training size 28709 : private val size 3589 : public val size 3589


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using %s for computation" % device)

Using cuda for computation


In [ ]:
time_2 = time.time()
print('Data processing takes time：',round((time_2 - time_1),2),'s')

Data processing takes time： 40.16 s


Model building

In [ ]:
import torch
from torch.utils import data
import numpy as np
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
import torchvision.models as models

In [ ]:
class Train_Dataset(data.Dataset):
    def __init__(self,X_train,labels):
        super(Train_Dataset,self).__init__()
        img = []
        label = []
        label = labels
        a = [train_preprocess(X_train[i])  for i in range(X_train.shape[0])]
        img = a
        self.img = img
        self.label=labels
      
            
    def __getitem__(self, index):
        
        imgs = self.img[index]
        labels = self.label[index]
        imgs_tensors =  imgs.type('torch.cuda.FloatTensor')
        return imgs_tensors, labels
        
    
    def __len__(self):
        return len(self.img)

In [ ]:
class Val_Dataset(data.Dataset):
    def __init__(self,X_val,labels):
        super(Val_Dataset,self).__init__()
        img = []
        label = []
        label = labels
        b = [val_preprocess(X_val[i])  for i in range(X_val.shape[0])]
        img = b
        self.img = img
        self.label=labels
      
             
    def __getitem__(self, index):
        
        imgs = self.img[index]
        labels = self.label[index]
        imgs_tensors =  imgs.type('torch.cuda.FloatTensor')
        return imgs_tensors, labels
        
    
    def __len__(self):
        return len(self.img)

In [ ]:
def validate_train(model,dataset,batch_size):
    val_loader = data.DataLoader(dataset,batch_size,shuffle=True)
    result,num = 0.0, 0
    for images,labels in val_loader:
        images = images.cuda()
        pre = model.forward(images)
        pre = pre.cpu()
        pre = np.argmax(pre.data.numpy(),axis = 1)
        labels = labels.data.numpy()
        result += np.sum((pre == labels))
        num += len(images)
    acc = result / num
    return acc

In [ ]:
def validate_val(model,dataset,batch_size):
    val_loader = data.DataLoader(dataset,batch_size,shuffle=True)
    result,num = 0.0, 0
    for images,labels in val_loader:
        for i in range(len(images)):
            images[i] = images[i].cuda()
            pre = model.forward(images[i])
            pre =pre.cpu()
            pre = np.argmax(pre.data.numpy().mean(0))
            if pre == labels[i] :
                result = result + 1
        num += len(images)
    acc = result / num
    return acc

Multi-model fusion(vgg19 + resnet18)

The first layer of the fusion model, training each base model（VGG19 + resnet18）

In [ ]:
train_dataset = X_train
train_labels = y_train
Val_dataset = Val_Dataset( X_val,y_val)
Test_dataset = Val_Dataset(X_test,y_test)

batch_size= 128
learning_rate = 0.001
epochs= 50

#resnet18
resnet_historyloss = []
resnet_historyacc = []
resnet_historytrac = []
resnet_historytestac = []

#vgg19
vgg19_historyloss = []
vgg19_historyacc = []
vgg19_historytrac = []
vgg19_historytestac = []

#multiple
multiple_historyloss = []
multiple_historyacc = []
multiple_historytrac = []
multiple_historytestac = []

batch_size = 128

In [ ]:
def train_resnet18(train_dataset,train_labels,Val_dataset,Test_dataset,batch_size,epochs,learning_rate,momen_tum,wt_decay):
    
# Build model
     resnet18 = models.resnet18()
     resnet18.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     resnet18.fc = torch.nn.Linear(in_features=512, out_features=7, bias=True)
    
     model = resnet18.cuda()
# Loss function
     loss_function = nn.CrossEntropyLoss()
     loss_function =  loss_function.cuda()
    
# Optimizer
     optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=momen_tum,weight_decay=wt_decay)

# Training
     print("Resnet18")
     for epoch in range(epochs):
# Each round of the input picture makes some changes to slow down the speed of overfitting
         Train_dataset = Train_Dataset(train_dataset,train_labels)
         train_loader = data.DataLoader(Train_dataset,batch_size,shuffle=True)
         loss_rate = 0
         model.train()
         for images,labels in train_loader:            
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            output = model.forward(images)
            loss_rate = loss_function(output,labels)
            loss_rate.backward()
            optimizer.step()
         resnet_historyloss.append(loss_rate.item())

# Print the loss per round 
         model.eval()
         acc_train = validate_train(model, Train_dataset, batch_size)
         resnet_historytrac.append(acc_train)

         acc_val = validate_val(model,Val_dataset,batch_size)
         resnet_historyacc.append(acc_val)

         acc_test = validate_val(model,Test_dataset,batch_size)
         resnet_historytestac.append(acc_test)
        
         if( (epoch+1) == epochs):
             print("Resnet18 Results：")
             print('The acc_train is :',acc_train)
             print('The acc_val is :',acc_val)
             print('The acc_test is :',acc_test)
             print('\n')

     
     print("Resnet18 Completed！")        
     return model

In [ ]:
def train_vgg19(train_dataset,train_labels,Val_dataset,Test_dataset,batch_size,epochs,learning_rate,momen_tum,wt_decay):
 
# Build model
     vgg19 = models.vgg19()
     vgg19.features[0] = torch.nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     vgg19.classifier[6] = torch.nn.Linear(in_features=4096, out_features=7, bias=True)
   
     model = vgg19.cuda()
# Loss function
     loss_function = nn.CrossEntropyLoss()
     loss_function =  loss_function.cuda()
    
# Optimizer
     optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=momen_tum,weight_decay=wt_decay)

# Training
     print("VGG19")
     for epoch in range(epochs):
#        Each round of the input picture makes some changes to slow down the speed of overfitting
         Train_dataset = Train_Dataset(train_dataset,train_labels)
         train_loader = data.DataLoader(Train_dataset,batch_size,shuffle=True)
         loss_rate = 0
         model.train()
         for images,labels in train_loader:
             images = images.cuda()
             labels = labels.cuda()            
             optimizer.zero_grad()
             output = model.forward(images)
             loss_rate = loss_function(output,labels)
             loss_rate.backward()
             optimizer.step()
         vgg19_historyloss.append(loss_rate.item())


         model.eval()
         acc_train = validate_train(model, Train_dataset, batch_size)
         vgg19_historytrac.append(acc_train)

         acc_val = validate_val(model,Val_dataset,batch_size)
         vgg19_historyacc.append(acc_val)

         acc_test = validate_val(model,Test_dataset,batch_size)
         vgg19_historytestac.append(acc_test)


         if((epoch+1) == epochs):
            
             print("VGG19 Results：")
             print('The acc_train is :',acc_train)
             print('The acc_val is :',acc_val)
             print('The acc_test is :',acc_test)
             print('\n')

     print("VGG19 Completed！")       
     return model

In [ ]:
resnet18 = train_resnet18(train_dataset,train_labels,Val_dataset,Test_dataset,batch_size,epochs,learning_rate ,momen_tum=0.9,wt_decay = 5e-4)
torch.save(resnet18,'fer2013_resnet18_model.pkl')

Resnet18
Resnet18 Results：
The acc_train is : 0.6853599916402522
The acc_val is : 0.5650599052660908
The acc_test is : 0.572025633881304


Resnet18 Completed！


In [ ]:
vgg19 = train_vgg19(train_dataset,train_labels,Val_dataset,Test_dataset,batch_size,epochs,learning_rate ,momen_tum=0.9,wt_decay = 5e-4)
torch.save(vgg19,'fer2013_vgg19_model.pkl')

VGG19
VGG19 Results：
The acc_train is : 0.5891880594935386
The acc_val is : 0.5589300640847032
The acc_test is : 0.566731680133742


VGG19 Completed！


Load the trained base model

In [5]:
resnet = torch.load("fer2013_resnet18_model.pkl")

In [ ]:
vgg = torch.load("fer2013_vgg19_model.pkl")

Build a fusion model network

In [ ]:
class Multiple(nn.Module):
    def __init__(self):
        super(Multiple,self).__init__()        
        
        self.fc = nn.Sequential(
             nn.Linear(in_features = 14,out_features = 7),
        )
        
    def forward(self,x):
        
        #After base model preprocessing
        result_1 = vgg(x)
        result_2 = resnet(x)
        
        #Features after splicing the base model
        result_1 = result_1.view(result_1.shape[0],-1)
        result_2 = result_2.view(result_2.shape[0],-1)
        result = torch.cat((result_1,result_2),1)
        
        #Input the features processed by the base model into the fusion model
        y = self.fc(result)
        
        return y

In [ ]:
def multiple_train(train_dataset,train_labels,Val_dataset,Test_dataset,batch_size,epochs,learning_rate,momen_tum,wt_decay):

    
# Build model
    model = Multiple()
    model = model.cuda()
    loss_function = nn.CrossEntropyLoss()
    loss_function =  loss_function.cuda()
    optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=momen_tum,weight_decay=wt_decay)

    print("Fusion model starts training！")
    for epoch in range(epochs):
        Train_dataset = Train_Dataset(train_dataset,train_labels)
        train_loader = data.DataLoader(Train_dataset,batch_size,shuffle=True)
        loss_rate = 0
        model.train()
        for images,labels in train_loader:         
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            output = model(images)
            loss_rate = loss_function(output,labels)
            loss_rate.backward()
            optimizer.step()
        multiple_historyloss.append(loss_rate.item())        
        
        model.eval()
        
        acc_train = validate_train(model, Train_dataset, batch_size)
        multiple_historytrac.append(acc_train)
        
        acc_val = validate_val(model,Val_dataset,batch_size)
        multiple_historyacc.append(acc_val)
        
        acc_test = validate_val(model,Test_dataset,batch_size)
        multiple_historytestac.append(acc_test)

        
        print('After {} epochs : '.format(epoch+1))
        print('The loss_rate is :',loss_rate.item())
        print('The acc_train is :',acc_train)
        print('The acc_val is :',acc_val)
        print('The acc_test is :',acc_test)
        print('\n')
    
    print("End of fusion model training！")   
    
    return model

Fusion Model Results


In [ ]:
model = multiple_train(train_dataset,train_labels,Val_dataset,Test_dataset,batch_size,epochs,learning_rate ,momen_tum=0.9,wt_decay = 5e-4)
torch.save(model,'fer2013_multiple_model.pkl')

Load the base model and the fusion model separately

In [ ]:
vgg = torch.load('fer2013_vgg19_model.pkl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set 

In [ ]:
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [7]:
model = torch.load('fer2013_resnet18_model.pkl')

In [8]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
model_mul = torch.load('fer2013_multiple_model.pkl')

AttributeError: ignored

Deploying using Flask

In [ ]:
import pandas as pd
import flask
import tensorflow as tf

In [ ]:

global graph

graph = tf.compat.v1.get_default_graph()

model = torch.load('fer2013_resnet18_model.pkl')
model.eval()
app = flask.Flask(__name__)
@app.route("/", methods=["GET","POST"])

def predict():
  data = {"success": False}
  
  params = flask.request.args
  
  if "G1" in params.keys():
    new_row = { "G1": params.get("G1"), "G2": params.get("G2"),
      "G3": params.get("G3"), "G4": params.get("G4"),
      "G5": params.get("G5"), "G6": params.get("G6"),
      "G7": params.get("G7"), "G8": params.get("G8"),
      "G9": params.get("G9"), "G10": params.get("G10") }
    new_x = pd.DataFrame.from_dict(new_row,
      orient = "index").transpose()
    
    with graph.as_default():
      data["response"] = str(model.eval(new_x)[0][0])
      data["success"] = True
  return flask.jsonify(data)
if __name__ == '__main__':
  app.run(host='0.0.0.0')

In [ ]:
%tb